# OmniQ Basic Usage Examples

This notebook demonstrates the basic usage of the OmniQ library for both synchronous and asynchronous task processing.

## Overview

OmniQ is a modular Python task queue library that supports:
- Multiple storage backends (File, Memory, SQLite, PostgreSQL, Redis, NATS)
- Both synchronous and asynchronous interfaces
- Task scheduling and recurring tasks
- Multiple worker types
- Task lifecycle event logging

Let's start with the basic examples.

## Setup and Imports

First, let's import the necessary modules:

In [ ]:
import asyncio
import datetime as dt
from omniq import OmniQ, AsyncOmniQ
from omniq.queue import FileTaskQueue
from omniq.storage import SQLiteResultStorage, PostgresEventStorage

## Synchronous Usage Example

The synchronous interface is perfect for traditional Python applications and scripts. Here's how to use it:

In [ ]:
# Create OmniQ instance with different storage backends
oq_sync = OmniQ(
    project_name="my_project",
    task_queue=FileTaskQueue(base_dir="some/path", queues=["low", "medium", "high"]),
    result_store=SQLiteResultStorage(base_dir="some/path"),
    event_store=PostgresEventStorage(host="localhost", port=5432, username="postgres")
)

print("Created synchronous OmniQ instance")

In [ ]:
# Define a simple task function
def simple_task(name):
    print(f"Hello {name}")
    return name

print("Defined simple_task function")

In [ ]:
# Start the worker
oq_sync.start_worker()
print("Worker started")

In [ ]:
# Enqueue a task with various options
task_id = oq_sync.enqueue(
    func=simple_task,
    func_args=dict(name="Tom"),
    queue_name="low",
    run_in=dt.timedelta(seconds=100),  # Delay execution
    ttl=dt.timedelta(hours=1),         # Task expires after 1 hour
    result_ttl=dt.timedelta(minutes=5) # Result expires after 5 minutes
)

print(f"Enqueued task with ID: {task_id}")

In [ ]:
# Get the result
result = oq_sync.get_result(task_id)
print(f"Task result: {result}")

In [ ]:
# Schedule a recurring task
schedule_id = oq_sync.schedule(
    func=simple_task,
    func_args=dict(name="Tom"),
    interval=dt.timedelta(seconds=10),  # Run every 10 seconds
    queue_name="low"
)

print(f"Scheduled recurring task with ID: {schedule_id}")

In [ ]:
# Get latest result from scheduled task
latest_result = oq_sync.get_result(schedule_id=schedule_id, kind="latest")
print(f"Latest scheduled task result: {latest_result}")

In [ ]:
# Stop the worker
oq_sync.stop_worker()
print("Worker stopped")

### Synchronous Context Manager Example

Using context managers ensures proper resource cleanup:

In [ ]:
# Using sync context manager for proper resource management
with OmniQ(
    project_name="my_project",
    task_queue=FileTaskQueue(base_dir="some/path", queues=["low", "medium", "high"]),
    result_store=SQLiteResultStorage(base_dir="some/path"),
    event_store=PostgresEventStorage(host="localhost", port=5432, username="postgres")
) as oq:
    task_id = oq.enqueue(simple_task, func_args=dict(name="Tom"))
    result = oq.get_result(task_id)
    print(f"Context manager result: {result}")

## Asynchronous Usage Example

The asynchronous interface is ideal for high-performance applications and when you need to handle many concurrent operations:

In [ ]:
# Create AsyncOmniQ instance
oq_async = AsyncOmniQ(
    project_name="my_project",
    task_queue=FileTaskQueue(base_dir="some/path", queues=["low", "medium", "high"]),
    result_store=SQLiteResultStorage(base_dir="some/path"),
    event_store=PostgresEventStorage(host="localhost", port=5432, username="postgres")
)

print("Created asynchronous OmniQ instance")

In [ ]:
# Define an async task function
async def async_task(name):
    print(f"Hello {name}")
    # Simulate some async work
    await asyncio.sleep(0.1)
    return name

print("Defined async_task function")

In [ ]:
# Async example function
async def run_async_example():
    # Start the worker
    await oq_async.start_worker()
    print("Async worker started")
    
    # Enqueue a task
    task_id = await oq_async.enqueue(
        func=async_task,
        func_args=dict(name="Tom"),
        queue_name="low",
        run_in=dt.timedelta(seconds=100),
        ttl=dt.timedelta(hours=1),
        result_ttl=dt.timedelta(minutes=5)
    )
    
    print(f"Enqueued async task with ID: {task_id}")
    
    # Get the result
    result = await oq_async.get_result(task_id)
    print(f"Async task result: {result}")
    
    # Schedule a recurring task
    schedule_id = await oq_async.schedule(
        func=async_task,
        func_args=dict(name="Tom"),
        interval=dt.timedelta(seconds=10),
        queue_name="low"
    )
    
    print(f"Scheduled recurring async task with ID: {schedule_id}")
    
    # Get latest result from scheduled task
    latest_result = await oq_async.get_result(schedule_id=schedule_id, kind="latest")
    print(f"Latest scheduled async task result: {latest_result}")
    
    # Stop the worker
    await oq_async.stop_worker()
    print("Async worker stopped")

# Run the async example
await run_async_example()

### Asynchronous Context Manager Example

Using async context managers for proper resource management:

In [ ]:
# Using async context manager
async def run_async_context_example():
    async with AsyncOmniQ(
        project_name="my_project",
        task_queue=FileTaskQueue(base_dir="some/path", queues=["low", "medium", "high"]),
        result_store=SQLiteResultStorage(base_dir="some/path"),
        event_store=PostgresEventStorage(host="localhost", port=5432, username="postgres")
    ) as oq:
        task_id = await oq.enqueue(async_task, func_args=dict(name="Tom"))
        result = await oq.get_result(task_id)
        print(f"Async context manager result: {result}")

# Run the async context manager example
await run_async_context_example()

## Key Concepts Explained

### Storage Backends

OmniQ uses three separate storage systems:

1. **Task Queue**: Stores pending tasks waiting to be executed
   - `FileTaskQueue`: Uses filesystem for task storage
   - Supports multiple named queues with priority ordering

2. **Result Storage**: Stores task execution results
   - `SQLiteResultStorage`: Uses SQLite database for result storage
   - Provides fast retrieval and querying capabilities

3. **Event Storage**: Logs task lifecycle events for monitoring
   - `PostgresEventStorage`: Uses PostgreSQL for event logging
   - Enables advanced querying and analysis of task execution

### Task Options

- **`queue_name`**: Specifies which queue to use (enables priority processing)
- **`run_in`**: Delays task execution by the specified time
- **`ttl`**: Task time-to-live - task expires if not executed within this time
- **`result_ttl`**: Result time-to-live - result is cleaned up after this time

### Scheduling

- **`interval`**: Creates recurring tasks that run at specified intervals
- **`schedule_id`**: Used to retrieve results from scheduled tasks
- **`kind="latest"`**: Gets the most recent result from a scheduled task

### Context Managers

Both sync and async versions support context managers for automatic resource cleanup:
- Automatically start/stop workers
- Ensure proper connection cleanup
- Handle exceptions gracefully

## Next Steps

This basic example shows the fundamental concepts of OmniQ. For more advanced usage, explore:

- **Component-based usage**: Create and configure individual components
- **Backend-based usage**: Use unified backends for all storage needs
- **Configuration-based usage**: Load settings from YAML files or environment variables
- **Multiple queues**: Implement priority-based task processing
- **Mixed sync/async tasks**: Handle both synchronous and asynchronous functions
- **Cloud storage**: Use S3, Azure, or GCP for distributed task processing

Check the other example directories for these advanced patterns!